In [1]:
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
dataset_name='citeseer'
args.dropout=0.0
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)
nodesize = features.shape[0]

# Some preprocessing
features_tmp=features.copy()
features = preprocess_features(features).A
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
# Setup Surrogate model
idx_train=np.array(np.where(train_mask==1)).tolist()[0]
idx_val=np.array(np.where(val_mask==1)).tolist()[0]
idx_unlabeled=np.array(np.where(test_mask==1)).tolist()[0]
surrogate = GCN(nfeat=features.shape[1], nclass=single_label.max().item()+1,
                nhid=256, dropout=0,lr=0.001, with_relu=True, with_bias=False, device='cpu').to('cpu')
print(type(features))
surrogate.fit(features, adj, single_label, idx_train, idx_val, patience=100)
# Setup Attack Model
model = Metattack(surrogate, nnodes=adj.shape[0], feature_shape=features.shape,
        attack_structure=True, attack_features=False, device='cpu', lambda_=0).to('cpu')
# Attack
ptb_rate = 0.03
perturbations = int(ptb_rate * (adj.sum()//2))


model.attack(features, adj, single_label, idx_train, idx_unlabeled, n_perturbations=perturbations, ll_constraint=False)
modified_adj = model.modified_adj
# print(adj)
# print("shiy")
# print(modified_adj)

2023-06-21 09:41:43.020316: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 09:41:43.242107: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-21 09:41:44.346687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-21 09:41:44.346799: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

<class 'numpy.ndarray'>


Perturbing graph:   0%|          | 0/138 [00:00<?, ?it/s]

GCN loss on unlabled data: 1.7543929815292358
GCN acc on unlabled data: 0.663
attack loss: 1.7481796741485596


Perturbing graph:   1%|          | 1/138 [03:15<7:26:06, 195.37s/it]

...

GCN loss on unlabled data: 1.7578752040863037
GCN acc on unlabled data: 0.633
attack loss: 1.751731276512146


Perturbing graph:   3%|▎         | 4/138 [13:05<7:19:51, 196.95s/it]

GCN loss on unlabled data: 1.7562236785888672
GCN acc on unlabled data: 0.644
attack loss: 1.7518296241760254


Perturbing graph:   4%|▎         | 5/138 [17:03<7:33:52, 204.76s/it]


KeyboardInterrupt: 

In [ ]:
modified_adj=sp.csr_array(modified_adj.int())

In [ ]:
print(type(modified_adj.toarray()))
np.save('tmp'+str(ptb_rate/0.01)+'.npy',modified_adj.toarray()) 

<class 'numpy.ndarray'>


In [ ]:
print(args.dropout)

0.0


In [ ]:
##PTDNEt

from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        #nuclear = model.my_nuclear()
        nuclear = model.nuclear()
        loss = cross_loss + args.weight_decay*lossL2 + args.lambda1*lossl0 + args.lambda3*nuclear + args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))


/home/zsp/ljx/PTDNet/utils.py:221: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
2023-06-20 23:30:23.060400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-06-20 23:30:23.646199: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55ed5abf1330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-20 23:30:23.646226: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-06-20 23:30:23.651527: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-20 23:30:23.784817: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the li

edge_vol 4687.5312
Epoch: 0001 train_loss= 1.79177 val_loss= 1.79114 train_acc= 0.40833 val_acc= 0.26400 best_val_acc_trail= 0.26400 test_acc= 0.26200
time  2.971179485321045
edge_vol 4675.9277
Epoch: 0002 train_loss= 1.78975 val_loss= 1.79047 train_acc= 0.56667 val_acc= 0.30600 best_val_acc_trail= 0.30600 test_acc= 0.31700
time  3.3047103881835938
edge_vol 4663.8853
Epoch: 0003 train_loss= 1.78768 val_loss= 1.78978 train_acc= 0.80000 val_acc= 0.36800 best_val_acc_trail= 0.36800 test_acc= 0.37200
time  3.5706982612609863
edge_vol 4651.415
Epoch: 0004 train_loss= 1.78564 val_loss= 1.78905 train_acc= 0.83333 val_acc= 0.39800 best_val_acc_trail= 0.39800 test_acc= 0.41700
time  3.835038423538208
edge_vol 4638.62
Epoch: 0005 train_loss= 1.78344 val_loss= 1.78826 train_acc= 0.86667 val_acc= 0.44600 best_val_acc_trail= 0.44600 test_acc= 0.45000
time  4.112331867218018
edge_vol 4625.558
Epoch: 0006 train_loss= 1.78126 val_loss= 1.78741 train_acc= 0.87500 val_acc= 0.47400 best_val_acc_trail= 0.

In [ ]:
## 我们的模型
from config import *
from utils import *
from models import GCN, PTDNetGCN
from metrics import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

# Settings
args.dataset=dataset_name
args.dropout=0.0


adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj
all_labels = y_train + y_test+y_val
single_label = np.argmax(all_labels,axis=-1)

nodesize = features.shape[0]

# Some preprocessing
features = preprocess_features(features)
support = preprocess_adj(adj)

optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)

tuple_adj = sparse_to_tuple(adj.tocoo())
features_tensor = tf.convert_to_tensor(features,dtype=dtype)
adj_tensor = tf.SparseTensor(*tuple_adj)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=dtype)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=dtype)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=dtype)
val_mask_tensor = tf.convert_to_tensor(val_mask)



best_test_acc = 0
best_val_acc_trail = 0
best_val_loss = 10000
import time
begin = time.time()

model = PTDNetGCN(input_dim=features.shape[1], output_dim=y_train.shape[1])
model.set_fea_adj(np.array(range(adj.shape[0])), features_tensor, adj_tensor)

best_epoch = 0
curr_step = 0
best_val_acc = 0

for epoch in range(args.epochs):
    temperature = max(0.05,args.init_temperature * pow(args.temperature_decay, epoch))
    with tf.GradientTape() as tape:
        preds = []
        for l in range(args.outL):
            output = model.call(temperature,training=True)
            preds.append(tf.expand_dims(output,0))
        all_preds = tf.concat(preds,axis=0)
        mean_preds = tf.reduce_mean(preds,axis=0)
        consistency_loss = tf.nn.l2_loss(mean_preds-all_preds)

        cross_loss = masked_softmax_cross_entropy(mean_preds, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        lossl0 = model.lossl0(temperature)
        nuclear = model.my_nuclear()
        #nuclear = model.nuclear()
        loss = cross_loss + args.lambda1*lossl0 + args.lambda3*nuclear+ args.weight_decay*lossL2 #+ args.coff_consis*consistency_loss
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call(None, training=False)
    edges_volumn = tf.reduce_sum(model.maskes[0])
    print('edge_vol',edges_volumn.numpy())

    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)
    if val_acc > best_val_acc:
        curr_step = 0
        best_epoch = epoch
        best_val_acc = val_acc
        best_val_loss= val_loss
        if val_acc>best_val_acc_trail:
            best_test_acc = test_acc
            best_val_acc_trail = val_acc
    else:
        curr_step +=1
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(train_acc), "val_acc=", "{:.5f}".format(val_acc),"best_val_acc_trail=", "{:.5f}".format(best_val_acc_trail),
      "test_acc=", "{:.5f}".format(best_test_acc))

    if curr_step > args.early_stop:
        print("Early stopping...")
        break
    end = time.time()
    print('time ',(end-begin))




edge_vol 4697.9355
Epoch: 0001 train_loss= 1.79149 val_loss= 1.79074 train_acc= 0.54167 val_acc= 0.33600 best_val_acc_trail= 0.33600 test_acc= 0.31900
time  1.8296616077423096
edge_vol 4691.7505
Epoch: 0002 train_loss= 1.78958 val_loss= 1.79010 train_acc= 0.77500 val_acc= 0.42200 best_val_acc_trail= 0.42200 test_acc= 0.40300
time  2.402817726135254
edge_vol 4684.5283
Epoch: 0003 train_loss= 1.78762 val_loss= 1.78941 train_acc= 0.89167 val_acc= 0.49400 best_val_acc_trail= 0.49400 test_acc= 0.48100
time  2.9739162921905518
edge_vol 4676.6523
Epoch: 0004 train_loss= 1.78561 val_loss= 1.78868 train_acc= 0.91667 val_acc= 0.53000 best_val_acc_trail= 0.53000 test_acc= 0.53700
time  3.6060988903045654
edge_vol 4668.3623
Epoch: 0005 train_loss= 1.78344 val_loss= 1.78789 train_acc= 0.93333 val_acc= 0.57000 best_val_acc_trail= 0.57000 test_acc= 0.56600
time  4.189894199371338
edge_vol 4659.7207
Epoch: 0006 train_loss= 1.78136 val_loss= 1.78702 train_acc= 0.93333 val_acc= 0.61400 best_val_acc_trai

In [ ]:
##GCN(被攻击）

from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name
args.dropout=0.0

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss# + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

0.0
Epoch: 0001 train_loss= 1.79190 val_loss= 1.79126 train_acc= 0.24200 val_acc= 0.24200 test_acc= 0.23900
Epoch: 0002 train_loss= 1.78946 val_loss= 1.79044 train_acc= 0.38800 val_acc= 0.38800 test_acc= 0.37200
Epoch: 0003 train_loss= 1.78690 val_loss= 1.78957 train_acc= 0.49200 val_acc= 0.49200 test_acc= 0.47500
Epoch: 0004 train_loss= 1.78424 val_loss= 1.78861 train_acc= 0.56800 val_acc= 0.56800 test_acc= 0.52000
Epoch: 0005 train_loss= 1.78143 val_loss= 1.78756 train_acc= 0.60400 val_acc= 0.60400 test_acc= 0.55500
Epoch: 0006 train_loss= 1.77844 val_loss= 1.78639 train_acc= 0.60800 val_acc= 0.60800 test_acc= 0.58000
Epoch: 0007 train_loss= 1.77523 val_loss= 1.78511 train_acc= 0.63200 val_acc= 0.63200 test_acc= 0.58800
Epoch: 0008 train_loss= 1.77178 val_loss= 1.78370 train_acc= 0.64200 val_acc= 0.64200 test_acc= 0.60600
Epoch: 0009 train_loss= 1.76811 val_loss= 1.78217 train_acc= 0.64800 val_acc= 0.64800 test_acc= 0.61500
Epoch: 0010 train_loss= 1.76421 val_loss= 1.78053 train_acc=

In [ ]:
##GCN_dropedge(0.2)


from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name
args.dropout=0.2

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

0.2
Epoch: 0001 train_loss= 1.79210 val_loss= 1.79151 train_acc= 0.21800 val_acc= 0.21800 test_acc= 0.19700
Epoch: 0002 train_loss= 1.79006 val_loss= 1.79123 train_acc= 0.29600 val_acc= 0.29600 test_acc= 0.23300
Epoch: 0003 train_loss= 1.79054 val_loss= 1.79094 train_acc= 0.33400 val_acc= 0.33400 test_acc= 0.28100
Epoch: 0004 train_loss= 1.78963 val_loss= 1.79063 train_acc= 0.36800 val_acc= 0.36800 test_acc= 0.31800
Epoch: 0005 train_loss= 1.78919 val_loss= 1.79031 train_acc= 0.39400 val_acc= 0.39400 test_acc= 0.35400
Epoch: 0006 train_loss= 1.78930 val_loss= 1.78999 train_acc= 0.43800 val_acc= 0.43800 test_acc= 0.38300
Epoch: 0007 train_loss= 1.78824 val_loss= 1.78966 train_acc= 0.46200 val_acc= 0.46200 test_acc= 0.42100
Epoch: 0008 train_loss= 1.78689 val_loss= 1.78931 train_acc= 0.49600 val_acc= 0.49600 test_acc= 0.45100
Epoch: 0009 train_loss= 1.78683 val_loss= 1.78895 train_acc= 0.52000 val_acc= 0.52000 test_acc= 0.46900
Epoch: 0010 train_loss= 1.78622 val_loss= 1.78858 train_acc=

In [ ]:
##GCN (未攻击)

from config import args
import tensorflow as tf
import time
from utils import *
from models import GCN_dropedge
from metrics import *

# Settings
optimizer = tf.keras.optimizers.Adam(learning_rate=args.lr)
args.dataset=dataset_name
args.dropout=0.0

adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)
#adj=modified_adj

tuple_adj = sparse_to_tuple(adj.tocoo())
adj_tensor = tf.SparseTensor(*tuple_adj)

features = preprocess_features(features)

model = GCN_dropedge(input_dim=features.shape[1], output_dim=y_train.shape[1], adj=adj_tensor)


features_tensor = tf.convert_to_tensor(features,dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train,dtype=tf.float32)
train_mask_tensor = tf.convert_to_tensor(train_mask)
y_test_tensor = tf.convert_to_tensor(y_test,dtype=tf.float32)
test_mask_tensor = tf.convert_to_tensor(test_mask)
y_val_tensor = tf.convert_to_tensor(y_val,dtype=tf.float32)
val_mask_tensor = tf.convert_to_tensor(val_mask)

best_test_acc = 0
best_val_acc = 0
best_val_loss = 10000


curr_step = 0
for epoch in range(args.epochs):

    with tf.GradientTape() as tape:
        output = model.call((features_tensor),training=True)
        cross_loss = masked_softmax_cross_entropy(output, y_train_tensor,train_mask_tensor)
        lossL2 = tf.add_n([tf.nn.l2_loss(v) for v in model.trainable_variables])
        loss = cross_loss# + args.weight_decay*lossL2
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    output = model.call((features_tensor), training=False)
    train_acc = masked_accuracy(output, y_train_tensor,train_mask_tensor)
    val_acc  = masked_accuracy(output, y_val_tensor,val_mask_tensor)
    val_loss = masked_softmax_cross_entropy(output, y_val_tensor, val_mask_tensor)
    test_acc  = masked_accuracy(output, y_test_tensor,test_mask_tensor)

    if val_acc > best_val_acc:
        curr_step = 0
        best_test_acc = test_acc
        best_val_acc = val_acc
        best_val_loss= val_loss
        # Print results

    else:
        curr_step +=1
    if curr_step > args.early_stop:
        print("Early stopping...")
        break

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cross_loss),"val_loss=", "{:.5f}".format(val_loss),
      "train_acc=", "{:.5f}".format(val_acc), "val_acc=", "{:.5f}".format(val_acc),
      "test_acc=", "{:.5f}".format(best_test_acc))

0.0
Epoch: 0001 train_loss= 1.79149 val_loss= 1.79074 train_acc= 0.37800 val_acc= 0.37800 test_acc= 0.35700
Epoch: 0002 train_loss= 1.78846 val_loss= 1.78982 train_acc= 0.48600 val_acc= 0.48600 test_acc= 0.46100
Epoch: 0003 train_loss= 1.78532 val_loss= 1.78882 train_acc= 0.54600 val_acc= 0.54600 test_acc= 0.52600
Epoch: 0004 train_loss= 1.78208 val_loss= 1.78773 train_acc= 0.60400 val_acc= 0.60400 test_acc= 0.57400
Epoch: 0005 train_loss= 1.77869 val_loss= 1.78651 train_acc= 0.62800 val_acc= 0.62800 test_acc= 0.60600
Epoch: 0006 train_loss= 1.77509 val_loss= 1.78517 train_acc= 0.65400 val_acc= 0.65400 test_acc= 0.62900
Epoch: 0007 train_loss= 1.77125 val_loss= 1.78370 train_acc= 0.66200 val_acc= 0.66200 test_acc= 0.64000
Epoch: 0008 train_loss= 1.76715 val_loss= 1.78209 train_acc= 0.67600 val_acc= 0.67600 test_acc= 0.65800
Epoch: 0009 train_loss= 1.76278 val_loss= 1.78035 train_acc= 0.68200 val_acc= 0.68200 test_acc= 0.66100
Epoch: 0010 train_loss= 1.75813 val_loss= 1.77849 train_acc=